In [104]:
from langgraph.graph import StateGraph,START, END
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from typing import TypedDict

In [105]:
load_dotenv()

True

In [106]:
model = ChatGoogleGenerativeAI(model='gemini-2.5-pro')


In [107]:
class BlogState(TypedDict):

    topic: str
    outline: str
    blog: str
    evaluate: int

In [108]:
def generate_outline(state:BlogState) -> BlogState:

    #Extract topic from the state
    topic = state['topic']

    #call llm for generation
    prompt = f'generate a 2 line outline for a blog from the topic - {topic}'

    outline = model.invoke(prompt).content

    state['outline'] = outline

    return state

In [109]:
def generate_blog(state: BlogState) -> BlogState:

    #Extract blog from the state
    topic = state['topic']
    outline = state['outline']

    #call llm for blog generation
    prompt = f'write a 5 line blog on the {topic} from the following \n {outline}'

    blog = model.invoke(prompt).content

    state['blog'] = blog

    return state

In [110]:
def evaluate(state: BlogState) -> BlogState:

    outline = state['outline']
    blog = state['blog']

    prompt = f'Based on the following outline \n {outline}. Rate my following blog \n {blog} and give me the score in integer out of 10'

    evaluation = model.invoke(prompt)
    score_str = evaluation.content
    try:
        score = int("".join([c for c in score_str if c.isdigit()]))
    except:
        score = 0  # fallback in case of unexpected response

    state['evaluate'] = score

    return state

In [111]:
#create our graph
graph = StateGraph(BlogState)

#Add node to our graph
graph.add_node('generate_outline', generate_outline)
graph.add_node('generate_blog', generate_blog)
graph.add_node('evaluate', evaluate)

#Add edge to our graph
graph.add_edge(START, 'generate_outline')
graph.add_edge('generate_outline', 'generate_blog')
graph.add_edge('generate_blog', 'evaluate')
graph.add_edge('evaluate', END)

workflow = graph.compile()

In [112]:
initial_state = {'topic': 'Rise of AI'}

final_state = workflow.invoke(initial_state)

print('blog Content', final_state['blog'])
print('Score', final_state['evaluate'])

blog Content Here is a 5-line blog post on the rise of AI, crafted from the provided outlines:

Once the stuff of science fiction, AI has now become a powerful part of our daily reality. This revolution is creating unprecedented opportunities while also forcing us to confront profound ethical dilemmas. It's a journey that is fundamentally reshaping our job market and challenging human creativity. As society shifts, our greatest task is to navigate the challenges and adapt our skills. To thrive in this new era, we must learn to work alongside our intelligent creations.
Score 9101231010510
